# Hierarchical Indexing

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects, respectively.
Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.
While Pandas does provide ``Panel`` and ``Panel4D`` objects that natively handle three-dimensional and four-dimensional data, a far more common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.
In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In this section, we'll explore the direct creation of ``MultiIndex`` objects, considerations when indexing, slicing, and computing statistics across multiply indexed data, and useful routines for converting between simple and hierarchically indexed representations of your data.

We begin with the standard imports:

Hasta este punto, nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas Series y DataFrame, respectivamente. A menudo es útil ir más allá y almacenar datos de dimensiones superiores, es decir, datos indexados por más de una o dos claves. Si bien Pandas proporciona objetos Panel y Panel4D que manejan de forma nativa datos tridimensionales y tetradimensionales, un patrón mucho más común en la práctica es utilizar la indexación jerárquica (también conocida como indexación múltiple) para incorporar múltiples niveles de índice dentro de un solo índice. De esta forma, los datos de dimensiones superiores se pueden representar de forma compacta dentro de la serie unidimensional familiar y los objetos de marco de datos bidimensionales.

En esta sección, exploraremos la creación directa de objetos MultiIndex, consideraciones al indexar, segmentar y calcular estadísticas a través de datos indexados múltiples, y rutinas útiles para convertir entre representaciones indexadas simples y jerárquicas de sus datos.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional ``Series``.
For concreteness, we will consider a series of data where each point has a character and numerical key.


Comencemos por considerar cómo podríamos representar datos bidimensionales dentro de una serie unidimensional. Para concretar, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### The bad way

Suppose you would like to track data about states from two different years.
Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:


Suponga que le gustaría rastrear datos sobre estados de dos años diferentes. Usando las herramientas de Pandas que ya hemos cubierto, puede sentirse tentado a usar simplemente las tuplas de Python como claves:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

Con este esquema de indexación, puede indexar o dividir directamente la serie en función de este índice múltiple:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:


Pero la conveniencia termina ahí. Por ejemplo, si necesita seleccionar todos los valores de 2010, deberá realizar algunos cambios desordenados (y potencialmente lentos) para que esto suceda:

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.


Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que nos ha encantado en Pandas.

### The Better Way: Pandas MultiIndex
Fortunately, Pandas provides a better way.
Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas ``MultiIndex`` type gives us the type of operations we wish to have.
We can create a multi-index from the tuples as follows:

Afortunadamente, Pandas ofrece una mejor manera. Nuestra indexación basada en tuplas es esencialmente un índice múltiple rudimentario, y el tipo Pandas MultiIndex nos da el tipo de operaciones que deseamos tener. Podemos crear un índice múltiple a partir de las tuplas de la siguiente manera:

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Notice that the ``MultiIndex`` contains multiple *levels* of indexing–in this case, the state names and the years, as well as multiple *labels* for each data point which encode these levels.

If we re-index our series with this ``MultiIndex``, we see the hierarchical representation of the data:

Observe que el MultiIndex contiene múltiples niveles de indexación, en este caso, los nombres de estado y los años, así como múltiples etiquetas para cada punto de datos que codifican estos niveles.

Si volvemos a indexar nuestra serie con este MultiIndex, vemos la representación jerárquica de los datos:

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the ``Series`` representation show the multiple index values, while the third column shows the data.
Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it.


Aquí las dos primeras columnas de la representación de la Serie muestran los múltiples valores del índice, mientras que la tercera columna muestra los datos. Observe que faltan algunas entradas en la primera columna: en esta representación de múltiples índices, cualquier entrada en blanco indica el mismo valor que la línea que se encuentra sobre ella

Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

The result is a singly indexed array with just the keys we're interested in.
This syntax is much more convenient (and the operation is much more efficient!) than the home-spun tuple-based multi-indexing solution that we started with.
We'll now further discuss this sort of indexing operation on hieararchically indexed data.


El resultado es una matriz indizada individualmente con solo las claves que nos interesan. Esta sintaxis es mucho más conveniente (¡y la operación es mucho más eficiente!) Que la solución de indexación múltiple basada en tuplas caseras con la que comenzamos. Ahora analizaremos más a fondo este tipo de operación de indexación en datos indexados jerárquicamente.


### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

MultiIndex como dimensión extra
Puede notar algo más aquí: podríamos haber almacenado fácilmente los mismos datos utilizando un simple DataFrame con etiquetas de índice y columna. De hecho, Pandas está construido con esta equivalencia en mente. El método unstack () convertirá rápidamente una Serie con índice múltiple en un Marco de Datos indexado convencionalmente:

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the ``stack()`` method provides the opposite operation:

In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Seeing this, you might wonder why would we would bother with hierarchical indexing at all.
The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional ``Series``, we can also use it to represent data of three or more dimensions in a ``Series`` or ``DataFrame``.
Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a ``MultiIndex`` this is as easy as adding another column to the ``DataFrame``:


Al ver esto, es posible que se pregunte por qué nos molestaría la indexación jerárquica.
La razón es simple: así como pudimos usar la indexación múltiple para representar datos bidimensionales dentro de una `` Serie '' unidimensional, también podemos usarla para representar datos de tres o más dimensiones en una `` Serie `` o `` DataFrame ''.
Cada nivel adicional en un índice múltiple representa una dimensión adicional de datos; Aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer agregar otra columna de datos demográficos para cada estado en cada año (por ejemplo, población menor de 18 años); con un `` MultiIndex '' esto es tan fácil como agregar otra columna al `` DataFrame '':



In [10]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In addition, all the ufuncs and other functionality discussed in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) work with hierarchical indices as well.
Here we compute the fraction of people under 18 by year, given the above data:

Además, todos los ufuncs y otras funcionalidades discutidas en Operación de datos en pandas también funcionan con índices jerárquicos. Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explore even high-dimensional data.


Esto nos permite manipular y explorar fácil y rápidamente incluso datos de alta dimensión

## Methods of MultiIndex Creation

The most straightforward way to construct a multiply indexed ``Series`` or ``DataFrame`` is to simply pass a list of two or more index arrays to the constructor. For example:


La forma más directa de construir una Serie o un Marco de Datos indexados es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo:

In [12]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.588620  0.842963
  2  0.028895  0.254439
b 1  0.050307  0.440760
  2  0.910613  0.413080

The work of creating the ``MultiIndex`` is done in the background.

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a ``MultiIndex`` by default:


El trabajo de crear el MultiIndex se realiza en segundo plano.

Del mismo modo, si pasa un diccionario con las tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un MultiIndex de forma predeterminada:

In [13]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Nevertheless, it is sometimes useful to explicitly create a ``MultiIndex``; we'll see a couple of these methods here.


Sin embargo, a veces es útil crear explícitamente un MultiIndex; veremos algunos de estos métodos aquí

### Explicit MultiIndex constructors

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the ``pd.MultiIndex``.
For example, as we did before, you can construct the ``MultiIndex`` from a simple list of arrays giving the index values within each level:


Constructores explícitos de MultiIndex
Para obtener más flexibilidad en cómo se construye el índice, en su lugar puede usar los constructores de métodos de clase disponibles en el pd.MultiIndex. Por ejemplo, como lo hicimos antes, puede construir el MultiIndex a partir de una simple lista de matrices que dan los valores de índice dentro de cada nivel:

In [14]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can construct it from a list of tuples giving the multiple index values of each point:


Puede construirlo a partir de una lista de tuplas que dan los múltiples valores de índice de cada punto:

In [15]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can even construct it from a Cartesian product of single indices:


Incluso puede construirlo a partir de un producto cartesiano de índices únicos:

In [16]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Similarly, you can construct the ``MultiIndex`` directly using its internal encoding by passing ``levels`` (a list of lists containing available index values for each level) and ``labels`` (a list of lists that reference these labels):


De manera similar, puede construir el MultiIndex directamente usando su codificación interna pasando niveles (una lista de listas que contienen valores de índice disponibles para cada nivel) y etiquetas (una lista de listas que hacen referencia a estas etiquetas):

In [17]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

TypeError: __new__() got an unexpected keyword argument 'labels'

Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.


Cualquiera de estos objetos se puede pasar como argumento de índice al crear una Serie o un Marco de datos, o se puede pasar al método de reindexar una Serie o un Marco de datos existente

### MultiIndex level names

Sometimes it is convenient to name the levels of the ``MultiIndex``.
This can be accomplished by passing the ``names`` argument to any of the above ``MultiIndex`` constructors, or by setting the ``names`` attribute of the index after the fact:


Nombres de nivel MultiIndex
A veces es conveniente nombrar los niveles del MultiIndex. Esto se puede lograr pasando el argumento de los nombres a cualquiera de los constructores MultiIndex anteriores, o estableciendo el atributo de nombres del índice después del hecho:

In [18]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

With more involved datasets, this can be a useful way to keep track of the meaning of various index values.


Con conjuntos de datos más involucrados, esta puede ser una forma útil de realizar un seguimiento del significado de varios valores de índice.

### MultiIndex for columns

In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.
Consider the following, which is a mock-up of some (somewhat realistic) medical data:


MultiIndex para columnas
En un DataFrame, las filas y columnas son completamente simétricas, y así como las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles. Considere lo siguiente, que es una maqueta de algunos datos médicos (algo realistas):

In [19]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      35.0  38.8  36.0  38.1  28.0  37.0
     2      43.0  35.6  22.0  37.0  28.0  39.1
2014 1      22.0  37.4  46.0  36.6  27.0  38.3
     2      46.0  37.1  21.0  36.0  37.0  38.3

Here we see where the multi-indexing for both rows and columns can come in *very* handy.
This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number.
With this in place we can, for example, index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:


Aquí vemos dónde la indexación múltiple para filas y columnas puede ser muy útil. Estos son datos fundamentalmente de cuatro dimensiones, donde las dimensiones son el sujeto, el tipo de medición, el año y el número de visita. Con esto en su lugar, podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener un DataFrame completo que contenga solo la información de esa persona:

In [20]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      36.0  38.1
     2      22.0  37.0
2014 1      46.0  36.6
     2      21.0  36.0

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!


¡Para registros complicados que contienen múltiples mediciones etiquetadas en múltiples ocasiones para muchos sujetos (personas, países, ciudades, etc.), el uso de filas y columnas jerárquicas puede ser extremadamente conveniente!

## Indexing and Slicing a MultiIndex

Indexing and slicing on a ``MultiIndex`` is designed to be intuitive, and it helps if you think about the indices as added dimensions.
We'll first look at indexing multiply indexed ``Series``, and then multiply-indexed ``DataFrame``s.


La indexación y el corte en un MultiIndex está diseñado para ser intuitivo, y ayuda si piensa en los índices como dimensiones adicionales. Primero veremos la indexación de series indexadas múltiples, y luego los marcos de datos indexados múltiples.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier:


Considere la serie de poblaciones estatales indexadas con múltiples índices que vimos anteriormente:

In [21]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:


Podemos acceder a elementos individuales indexando con múltiples términos:

In [22]:
pop['California', 2000]

33871648

The ``MultiIndex`` also supports *partial indexing*, or indexing just one of the levels in the index.
The result is another ``Series``, with the lower-level indices maintained:

El MultiIndex también admite indexación parcial, o indexar solo uno de los niveles en el índice. El resultado es otra serie, con los índices de nivel inferior mantenidos:

In [23]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the ``MultiIndex`` is sorted (see discussion in [Sorted and Unsorted Indices](#Sorted-and-unsorted-indices)):


El corte parcial también está disponible, siempre y cuando el MultiIndex esté ordenado (vea la discusión en Índices ordenados y no ordenados):

In [24]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:
Con índices ordenados, la indexación parcial se puede realizar en niveles inferiores pasando un segmento vacío en el primer índice:

In [25]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Other types of indexing and selection (discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb)) work as well; for example, selection based on Boolean masks:

También funcionan otros tipos de indexación y selección (discutidos en Indización y selección de datos); por ejemplo, selección basada en máscaras booleanas:

In [26]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:


La selección basada en indexación elegante también funciona:

In [27]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:

Un DataFrame indexado múltiple se comporta de manera similar. Considere nuestro DataFrame médico de juguete de antes:

In [28]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      35.0  38.8  36.0  38.1  28.0  37.0
     2      43.0  35.6  22.0  37.0  28.0  39.1
2014 1      22.0  37.4  46.0  36.6  27.0  38.3
     2      46.0  37.1  21.0  36.0  37.0  38.3

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.
For example, we can recover Guido's heart rate data with a simple operation:

Recuerde que las columnas son primarias en un DataFrame, y la sintaxis utilizada para las Series con índice múltiple se aplica a las columnas. Por ejemplo, podemos recuperar los datos de frecuencia cardíaca de Guido con una operación simple:

In [29]:
health_data['Guido', 'HR']

year  visit
2013  1        36.0
      2        22.0
2014  1        46.0
      2        21.0
Name: (Guido, HR), dtype: float64

Also, as with the single-index case, we can use the ``loc``, ``iloc``, and ``ix`` indexers introduced in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb). For example:


Además, como con el caso de índice único, podemos usar los indexadores loc, iloc e ix introducidos en la Indización y selección de datos. Por ejemplo:

In [30]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      35.0  38.8
     2      43.0  35.6

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

Estos indexadores proporcionan una vista en forma de matriz de los datos bidimensionales subyacentes, pero a cada índice individual en loc o iloc se le puede pasar una tupla de múltiples índices. Por ejemplo

In [31]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        35.0
      2        43.0
2014  1        22.0
      2        46.0
Name: (Bob, HR), dtype: float64

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:

Trabajar con sectores dentro de estas tuplas de índice no es especialmente conveniente; intentar crear un segmento dentro de una tupla generará un error de sintaxis:

In [32]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-32-fb34fa30ac09>, line 1)

You could get around this by building the desired slice explicitly using Python's built-in ``slice()`` function, but a better way in this context is to use an ``IndexSlice`` object, which Pandas provides for precisely this situation.
For example:

Trabajar con sectores dentro de estas tuplas de índice no es especialmente conveniente; intentar crear un segmento dentro de una tupla generar un error de sintaxis:

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,35.0,36.0,28.0
2014,1,22.0,46.0,27.0


There are so many ways to interact with data in multiply indexed ``Series`` and ``DataFrame``s, and as with many tools in this book the best way to become familiar with them is to try them out!


¡Hay tantas maneras de interactuar con los datos en Series y marcos de datos indexados múltiples, y como con muchas herramientas en este libro, la mejor manera de familiarizarse con ellos es probarlos!

## Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.
There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations.
We saw a brief example of this in the ``stack()`` and ``unstack()`` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.


Una de las claves para trabajar con datos indexados múltiples es saber cómo transformar los datos de manera efectiva. Hay una serie de operaciones que conservarán toda la información en el conjunto de datos, pero la reorganizarán para varios cálculos. Vimos un breve ejemplo de esto en los métodos stack () y unstack (), pero hay muchas más formas de controlar finamente la reorganización de datos entre índices jerárquicos y columnas, y los exploraremos aquí.

### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:


Índices ordenados y sin clasificar
Anteriormente, mencionamos brevemente una advertencia, pero deberíamos enfatizarla más aquí. Muchas de las operaciones de corte de MultiIndex fallarán si el índice no está ordenado. Echemos un vistazo a esto aquí.

Comenzaremos creando algunos datos simples indexados de forma múltiple donde los índices no están ordenados lexográficamente:


In [34]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.366154
      2      0.285835
c     1      0.104368
      2      0.907040
b     1      0.884168
      2      0.583420
dtype: float64

If we try to take a partial slice of this index, it will result in an error:
Si intentamos tomar una porción parcial de este índice, se producirá un error

In [35]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted.
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.
Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.
We'll use the simplest, ``sort_index()``, here:


Aunque no está del todo claro en el mensaje de error, este es el resultado de que MultiIndex no se haya ordenado. Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles en el MultiIndex estén ordenados (es decir, lexográficos). Pandas proporciona una serie de rutinas de conveniencia para realizar este tipo de clasificación; ejemplos son los métodos sort_index () y sortlevel () del DataFrame. Usaremos el más simple, sort_index (), aquí:

In [36]:
data = data.sort_index()
data

char  int
a     1      0.366154
      2      0.285835
b     1      0.884168
      2      0.583420
c     1      0.104368
      2      0.907040
dtype: float64

In [37]:
With the index sorted in this way, partial slicing will work as expected:


Con el índice ordenado de esta manera, el corte parcial funcionará como se esperaba:

SyntaxError: invalid syntax (<ipython-input-37-f96c44c6b6c5>, line 1)

In [38]:
data['a':'b']

char  int
a     1      0.366154
      2      0.285835
b     1      0.884168
      2      0.583420
dtype: float64

### Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

Apilando y desapilando índices
Como vimos brevemente antes, es posible convertir un conjunto de datos de un índice múltiple apilado a una representación bidimensional simple, opcionalmente especificando el nivel a usar:

In [39]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [40]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of ``unstack()`` is ``stack()``, which here can be used to recover the original series:


Lo opuesto a unstack () es stack (), que aquí puede usarse para recuperar la serie original

In [41]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.
Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.
For clarity, we can optionally specify the name of the data for the column representation:


Ajuste y reinicio del índice
Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; Esto se puede lograr con el método reset_index. Llamar a esto en el diccionario de población dará como resultado un DataFrame con una columna de estado y año que contiene la información que anteriormente estaba en el índice. Para mayor claridad, opcionalmente podemos especificar el nombre de los datos para la representación de la columna:

In [42]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:


A menudo, cuando se trabaja con datos en el mundo real, los datos de entrada sin procesar se ven así y es útil construir un MultiIndex a partir de los valores de las columnas. Esto se puede hacer con el método set_index del DataFrame, que devuelve un DataFrame indexado de forma múltiple:


In [43]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In practice, I find this type of reindexing to be one of the more useful patterns when encountering real-world datasets.


En la práctica, considero que este tipo de reindexación es uno de los patrones más útiles cuando se encuentran conjuntos de datos del mundo real.

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:


Agregaciones de datos en múltiples índices
Anteriormente hemos visto que Pandas tiene métodos de agregación de datos integrados, como mean (), sum () y max (). Para los datos indexados jerárquicamente, se les puede pasar un parámetro de nivel que controla en qué subconjunto de los datos se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [44]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      35.0  38.8  36.0  38.1  28.0  37.0
     2      43.0  35.6  22.0  37.0  28.0  39.1
2014 1      22.0  37.4  46.0  36.6  27.0  38.3
     2      46.0  37.1  21.0  36.0  37.0  38.3

Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:


Quizás nos gustaría promediar las mediciones en las dos visitas cada año. Podemos hacer esto nombrando el nivel de índice que nos gustaría explorar, en este caso el año:

In [45]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     39.0  37.20  29.0  37.55  28.0  38.05
2014     34.0  37.25  33.5  36.30  32.0  38.30

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:
Al seguir utilizando la palabra clave del eje, también podemos tomar la media entre los niveles en las columnas:

In [46]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,32.000000,37.600000
2014,33.166667,37.283333


Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits each year.
This syntax is actually a short cut to the ``GroupBy`` functionality, which we will discuss in [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb).
While this is a toy example, many real-world datasets have similar hierarchical structure.

Por lo tanto, en dos líneas, hemos podido encontrar la frecuencia cardíaca y la temperatura promedio medidas entre todos los sujetos en todas las visitas cada año. Esta sintaxis es en realidad un atajo a la funcionalidad GroupBy, que discutiremos en Agregación y Agrupación. Si bien este es un ejemplo de juguete, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.

## Aside: Panel Data

Pandas has a few other fundamental data structures that we have not yet discussed, namely the ``pd.Panel`` and ``pd.Panel4D`` objects.
These can be thought of, respectively, as three-dimensional and four-dimensional generalizations of the (one-dimensional) ``Series`` and (two-dimensional) ``DataFrame`` structures.
Once you are familiar with indexing and manipulation of data in a ``Series`` and ``DataFrame``, ``Panel`` and ``Panel4D`` are relatively straightforward to use.
In particular, the ``ix``, ``loc``, and ``iloc`` indexers discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) extend readily to these higher-dimensional structures.

We won't cover these panel structures further in this text, as I've found in the majority of cases that multi-indexing is a more useful and conceptually simpler representation for higher-dimensional data.
Additionally, panel data is fundamentally a dense data representation, while multi-indexing is fundamentally a sparse data representation.
As the number of dimensions increases, the dense representation can become very inefficient for the majority of real-world datasets.
For the occasional specialized application, however, these structures can be useful.
If you'd like to read more about the ``Panel`` and ``Panel4D`` structures, see the references listed in [Further Resources](03.13-Further-Resources.ipynb).


Aparte: Datos del panel
Pandas tiene algunas otras estructuras de datos fundamentales que aún no hemos discutido, a saber, los objetos pd.Panel y pd.Panel4D. Estos pueden considerarse, respectivamente, como generalizaciones tridimensionales y tetradimensionales de las estructuras de la serie (unidimensional) y del marco de datos (bidimensional). Una vez que esté familiarizado con la indexación y la manipulación de datos en una Serie y un Marco de datos, Panel y Panel4D son relativamente fáciles de usar. En particular, los indexadores ix, loc e iloc discutidos en Data Indexing and Selection se extienden fácilmente a estas estructuras de dimensiones superiores.

No cubriremos más estas estructuras de panel en este texto, como he encontrado en la mayoría de los casos que la indexación múltiple es una representación más útil y conceptualmente más simple para datos de dimensiones superiores. Además, los datos del panel son fundamentalmente una representación de datos densa, mientras que la indexación múltiple es fundamentalmente una representación de datos dispersa. A medida que aumenta el número de dimensiones, la representación densa puede volverse muy ineficiente para la mayoría de los conjuntos de datos del mundo real. Sin embargo, para la aplicación especializada ocasional, estas estructuras pueden ser útiles. Si desea leer más sobre las estructuras Panel y Panel4D, consulte las referencias que figuran en Recursos adicionales.